# Aggregieren der Daten
In diesem Notebook werden die zuvor erstellten Datenquellen eingelesen und  zudem werden die Quelldaten mit zusätzlichen Daten angereichert um das Dataset zu vervollständigen und zur visualisierung bereitzustellen <br>
* **Sonnenscheindauer:** Einige Kantone fehlen daher muss die Sonnenscheindauer   zusätzlich über eine API abgerufen werden.
* **Mietpreise:** Es gibt verschiedene Wohnungsgrössen. Es muss sichergestellt sein, dass die Wohnungsgrössen einfach mit den entsprechenden Strompreisen vergleichbar sind. <br>
* **Strompreise:** Es gibt standard und reduzierte Strompreise. Es muss sichergestellt sein, dass auf eine passende grösse festgelegt wird, oder die unterschiedlichen Preis zumindest einfach zugägnlich für Vergleiche sind.<br>

# Installation notwendiger Bibliotheken

In [1]:
!pip install --upgrade pip
!pip install matplotlib
!pip install pandas
!pip install numpy
!pip install beautifulsoup4
!pip install openpyxl
!pip install overpy
!pip install osmium overpy
!wget https://download.geofabrik.de/europe/switzerland-latest.osm.pbf -O switzerland.osm.pbf

!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install meteostat #for https://dev.meteostat.net/python/


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:00
--2024-12-07 16:44:53--  https://download.geofabrik.de/europe/switzerland-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 65.109.48.72, 65.109.50.43, 2a01:4f9:5a:2797::2, ...
Connecting to download.geofabrik.de (download.geofabrik.de)|65.109.48.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 481199744 (459M) [application/octet-stream]
Saving to: ‘switzerland.osm.pbf’

switzerland.osm.pbf 100%[===================>] 458.91M  16.9MB/s    in 27s     

2024-12-07 16:45:20 (16.9 MB/s) - ‘switzerland.osm.pbf’ saved [481199744/481199744]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.0 MB/s eta 0:00:00


# Module importieren

In [2]:
import requests
import pandas as pd
import folium
import numpy as np
from pathlib import Path
from google.colab import drive # Library to access shared folder "ie_scripting_datasets" on google drive
import os
from bs4 import BeautifulSoup
import urllib.request
import openpyxl
from geopy.geocoders import Nominatim
import overpy
import json
from IPython.display import display
import datetime
import matplotlib.pyplot as plt
import meteostat
from datetime import datetime

# Speicher Ort für Datenquellen auf Google Drive

In [3]:
drive.mount('/content/drive') #will give access to "ie_scripting_datasets" on google drive in folder selection


Mounted at /content/drive


# Erstellung Globaler Dataframe für Sonnenscheindauer

In [4]:
#Dataframe with sunhours combined for all years

def create_sunshine_data():

  # Specify the path to your Excel file
  excel_file = '/content/drive/MyDrive/ie_scripting_datasets/Sonnenscheindauer.xlsx'

  # Read all sheets in the Excel file
  dfs = pd.read_excel(excel_file, sheet_name=None)

  # `dfs` is a dictionary where keys are sheet names and values are DataFrames

  # Initialize an empty list to store DataFrames
  df_list = []

  # Iterate through the dictionary items
  for sheet_name, df in dfs.items():
      # Optionally add a column to indicate the sheet name (e.g., the year)
      df['Year'] = sheet_name

      # Append the DataFrame to the list
      df_list.append(df)

  # Concatenate all DataFrames into a single DataFrame
  sunshine_data = pd.concat(df_list, ignore_index=True)

  # Return the combined DataFrame
  return sunshine_data
# Print the combined DataFrame
  #print(sunshine_data)


In [5]:
sunshine_data = create_sunshine_data()
#print(sunshine_data)

#Abfrage aller Standorte von Messstationen

In [6]:
def get_unique_stations(df):
  """
  Extracts unique values from the first column of a Pandas DataFrame and returns them as a set.

  Args:
    df: The Pandas DataFrame.

  Returns:
    A set containing the unique values from the first column.
  """

  # Get the first column (index 0) as a Series
  all_stations = sunshine_data.iloc[:, 0]

  # Convert the Series to a set to get unique values
  stations = set(all_stations)

  return stations

# Assuming you have already loaded your data into 'df_sonnenscheindauer_all_years'
# and called df_sonnenscheindauer_all_years() like in the previous solution:
# sonnenscheindauer_all_years_df = df_sonnenscheindauer_all_years()

# Now get the unique values from the second column:
stations = get_unique_stations(sunshine_data)

# Print the set
print(stations)

{'Pilatus', 'La Dôle', 'Adelboden', 'Davos', 'Lugano', 'Zürich / Kloten', 'St. Gallen', 'Disentis / Sedrun', 'Genève-Cointrin', 'Visp', 'Basel-Binningen', 'Zermatt', 'Chur', 'Neuchâtel', 'Interlaken', 'Engelberg', 'Luzern', 'Säntis', 'Pully', 'Scuol', 'Aigle', 'Zürich / Fluntern', 'Napf', 'La Chaux-de-Fonds', 'Sion', 'Jungfraujoch Sphinx', 'Aadorf / Tänikon', 'Rünenberg', 'Weissfluhjoch', 'Samedan-Flugplatz', 'Montana', 'Glarus', 'Buchs-Aarau', 'Güttingen', 'Vaduz', 'Gütsch ob Andermatt', 'San Bernardino', 'Fahy', 'Altdorf', 'Stabio', 'Locarno / Monti', 'Cimetta', 'Wynau', 'Payerne', 'Col du Gd-St-Bernard', 'Poschiavo / Robbia', 'Bern / Zollikofen', 'Schaffhausen', 'Wädenswil', 'Le Moléson'}


#Erstellen einer Liste aller Stationen

In [7]:
#saving the stations in a variable

stations = get_unique_stations(sunshine_data)
print(stations)

{'Pilatus', 'La Dôle', 'Adelboden', 'Davos', 'Lugano', 'Zürich / Kloten', 'St. Gallen', 'Disentis / Sedrun', 'Genève-Cointrin', 'Visp', 'Basel-Binningen', 'Zermatt', 'Chur', 'Neuchâtel', 'Interlaken', 'Engelberg', 'Luzern', 'Säntis', 'Pully', 'Scuol', 'Aigle', 'Zürich / Fluntern', 'Napf', 'La Chaux-de-Fonds', 'Sion', 'Jungfraujoch Sphinx', 'Aadorf / Tänikon', 'Rünenberg', 'Weissfluhjoch', 'Samedan-Flugplatz', 'Montana', 'Glarus', 'Buchs-Aarau', 'Güttingen', 'Vaduz', 'Gütsch ob Andermatt', 'San Bernardino', 'Fahy', 'Altdorf', 'Stabio', 'Locarno / Monti', 'Cimetta', 'Wynau', 'Payerne', 'Col du Gd-St-Bernard', 'Poschiavo / Robbia', 'Bern / Zollikofen', 'Schaffhausen', 'Wädenswil', 'Le Moléson'}


#Import aller Koordinaten der Stationen mittels Nomatim im zuordnung von Messtation to Kanton vorzubereiten

In [8]:


# Create a geolocator object
geolocator = Nominatim(user_agent="station_coordinate_finder")  # Replace "my_geocoder" with a descriptive name

# Get unique station names from the DataFrame
station_names = stations  # Replace 'Station Name' with your column name

station_coordinates = {}  # Initialize a dictionary to store coordinates

for station_name in station_names:
    try:
        # Geocode the station name
        location = geolocator.geocode(station_name + ", Switzerland")  # Add ", Switzerland" for better results

        # If location is found, store coordinates
        if location:
            station_coordinates[station_name] = (location.latitude, location.longitude)
            print(f"Coordinates for {station_name}: {location.latitude}, {location.longitude}")
        else:
            print(f"Could not find coordinates for {station_name}")

    except Exception as e:
        print(f"Error geocoding {station_name}: {e}")

print("\nStation Coordinates:")
print(station_coordinates)

Coordinates for Pilatus: 46.9706835, 8.398695918732447
Coordinates for La Dôle: 46.4256492, 6.0995546
Coordinates for Adelboden: 46.4927213, 7.5587618
Coordinates for Davos: 46.796198, 9.8236892
Coordinates for Lugano: 46.0050102, 8.9520281
Coordinates for Zürich / Kloten: 47.443892950000006, 8.571472097092306
Coordinates for St. Gallen: 47.425618, 9.3762397
Coordinates for Disentis / Sedrun: 46.703001900000004, 8.8186489317436
Coordinates for Genève-Cointrin: 46.225539299999994, 6.100365627759709
Coordinates for Visp: 46.2913113, 7.8811918
Coordinates for Basel-Binningen: 47.541255199999995, 7.582376595957297
Coordinates for Zermatt: 46.0212076, 7.749254
Coordinates for Chur: 46.854747, 9.5264904
Coordinates for Neuchâtel: 46.9895828, 6.9292641
Coordinates for Interlaken: 46.6855231, 7.8585139
Coordinates for Engelberg: 46.8223497, 8.4043996
Coordinates for Luzern: 47.0505452, 8.3054682
Coordinates for Säntis: 47.2493844, 9.3432366
Coordinates for Pully: 46.5098019, 6.6617476
Coordina

#Stationskoordinaten in ein JSON-File speichern

In [9]:
#check of station coordinates
len(station_coordinates)
# --> No coordinates for 2 Stations (Total = 50)

48

In [10]:
import json

# Assuming you have the station_coordinates dictionary

# Path to save the JSON file
file_path = '/content/drive/MyDrive/ie_scripting_datasets/station_coordinates.json'

# Convert station_coordinates to the desired format
data_to_save = {station_name: f"{location[0]}, {location[1]}" for station_name, location in station_coordinates.items()}

# Save the data to the JSON file with UTF-8 encoding
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(data_to_save, f, indent=4, ensure_ascii=False)  # Ensure UTF-8 and non-ASCII characters

print(f"Station coordinates saved to: {file_path}")

Station coordinates saved to: /content/drive/MyDrive/ie_scripting_datasets/station_coordinates.json


#Plotten der Stationen auf einer Schweizer Karte

In [11]:
import folium


# Create a base map centered on Switzerland
swiss_map = folium.Map(location=[46.8, 8.2], zoom_start=8)

# Add markers for each station
for station, coordinates in station_coordinates.items():
    folium.Marker(
        location=coordinates,
        popup=station,
        icon=folium.Icon(color="blue"),
    ).add_to(swiss_map)

# Display the map
display(swiss_map)


#Import einer Liste mit allen Gemeindenamen und Kantonszugehörigkeit mittels API


API Source: https://www.agvchapp.bfs.admin.ch/de#:~:text=Gemeinde%20abgefragt%20werden.-,Gemeindestand,-XLSX%2001.01.2024

In [12]:
"""
# import aller gemeinden und Erstellung Dataframe um Gemeinden an Kantone zuweisen zu können
file_url = 'https://www.agvchapp.bfs.admin.ch/de#:~:text=Gemeinde%20abgefragt%20werden.-,Gemeindestand,-XLSX%2001.01.2024'

# Load the Excel file from the URL into a DataFrame
df_gemeinden = pd.read_excel(file_url)

# Display the first few rows of the DataFrame
df_gemeinden.head()

print(df_gemeinden)

"""
import requests
import pandas as pd
import warnings
def get_all_municipalities():
  # Suppress the specific warning about the default style in openpyxl
  warnings.simplefilter("ignore", UserWarning)

  # Define the URL for the POST request (replace with the exact form action URL)
  url = "https://www.agvchapp.bfs.admin.ch/de/state/results/xls?SnapshotDate=01.01.2024"

  # Send the POST request to download the file containing all swiss municipalities
  response = requests.post(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Save the content as an Excel file locally
      with open("gemeinden.xlsx", "wb") as file:
          file.write(response.content)

      # Load the downloaded Excel file into a DataFrame
      try:
          df_gemeinden = pd.read_excel("gemeinden.xlsx", engine='openpyxl')
          # Display the first few rows of the DataFrame
          display(df_gemeinden.head())
      except Exception as e:
          print("An error occurred while reading the Excel file:", e)
  else:
      print("Failed to download the file. Status code:", response.status_code)

  return df_gemeinden





In [13]:
#Definieren aller gemeinden in einem Dataframe
gemeinden = get_all_municipalities()
len(gemeinden)

,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,13256,ZH,101,Affoltern,1,Aeugst am Albis,1976-11-15
1,11742,ZH,101,Affoltern,2,Affoltern am Albis,1848-09-12
2,11801,ZH,101,Affoltern,3,Bonstetten,1848-09-12
3,11992,ZH,101,Affoltern,4,Hausen am Albis,1911-01-01
4,12249,ZH,101,Affoltern,5,Hedingen,1848-09-12


2131

In [14]:
#Erstellen eines sets aller Kantone und Gemeinden
all_cantons = set(gemeinden.iloc[:, 1])
print("Total Cantons: ", len(all_cantons))
all_municipalities = set(gemeinden.iloc[:, 5])
print("Total Municipalities: ", len(all_municipalities))




Total Cantons:  26
Total Municipalities:  2131


#Speichern aller Gemeinten in JSON File für spätere Verwendung

In [15]:
# Assuming you have the 'gemeinden' DataFrame

# Path to save the JSON file on your Google Drive
file_path = '/content/drive/MyDrive/ie_scripting_datasets/gemeinden.json'  # Change 'gemeinden.json' if needed

# Save the DataFrame to JSON with UTF-8 encoding
# Remove the 'encoding' parameter, ensure force_ascii=False
gemeinden.to_json(file_path, orient='records', force_ascii=False, indent=4)

print(f"DataFrame 'gemeinden' saved to: {file_path}")

DataFrame 'gemeinden' saved to: /content/drive/MyDrive/ie_scripting_datasets/gemeinden.json


In [16]:
#Erstellen der Listen aller Stationen und Kantone

stations_list = list(stations)
cantons_list = list(all_cantons)

print(stations_list)
print(cantons_list)

['Pilatus', 'La Dôle', 'Adelboden', 'Davos', 'Lugano', 'Zürich / Kloten', 'St. Gallen', 'Disentis / Sedrun', 'Genève-Cointrin', 'Visp', 'Basel-Binningen', 'Zermatt', 'Chur', 'Neuchâtel', 'Interlaken', 'Engelberg', 'Luzern', 'Säntis', 'Pully', 'Scuol', 'Aigle', 'Zürich / Fluntern', 'Napf', 'La Chaux-de-Fonds', 'Sion', 'Jungfraujoch Sphinx', 'Aadorf / Tänikon', 'Rünenberg', 'Weissfluhjoch', 'Samedan-Flugplatz', 'Montana', 'Glarus', 'Buchs-Aarau', 'Güttingen', 'Vaduz', 'Gütsch ob Andermatt', 'San Bernardino', 'Fahy', 'Altdorf', 'Stabio', 'Locarno / Monti', 'Cimetta', 'Wynau', 'Payerne', 'Col du Gd-St-Bernard', 'Poschiavo / Robbia', 'Bern / Zollikofen', 'Schaffhausen', 'Wädenswil', 'Le Moléson']
['JU', 'ZG', 'AI', 'SZ', 'SH', 'AR', 'TG', 'GL', 'NE', 'AG', 'SG', 'LU', 'GR', 'VD', 'VS', 'BS', 'BE', 'BL', 'FR', 'GE', 'ZH', 'OW', 'NW', 'UR', 'SO', 'TI']


#Mapping der Stationsliste mit Gemeindeliste zur Identifikaton von fehlenden Einträgen



In [17]:
 """
  Compares a list of stations against a municipality DataFrame and returns a dictionary
  mapping station names to their cantons.

  Args:
    stations_list: A list of station names.
    gemeinden: A Pandas DataFrame containing municipality information,
              including columns for municipality name ('Gemeindename') and canton ('Kanton').

  Returns:
    A dictionary where keys are station names and values are their corresponding cantons.
  """

def get_station_canton_mapping(stations_list, gemeinden):

  station_canton_map = {}  # Initialize an empty dictionary

  for station in stations_list:
    # Find the row in 'gemeinden' where 'Gemeindename' matches the station
    matching_row = gemeinden[gemeinden['Gemeindename'] == station]

    # If a match is found, extract the canton and add to the dictionary
    if not matching_row.empty:
      canton = matching_row['Kanton'].iloc[0]  # Get canton from the first matching row
      station_canton_map[station] = canton
    else:
      """
      If no match is found, add with value "not found" / Kann auskommentiert werden um
      nur diejenigen stationen zu zeigen die effektif einem Kanton zugeordnet werden könnnen
      """
      station_canton_map[station] = "not found"


  return station_canton_map



In [18]:
# Erstellung des Dictionary für das Stations/Kantons Mapping
station_canton_map = get_station_canton_mapping(stations_list, gemeinden)
print(station_canton_map)
len(station_canton_map)

{'Pilatus': 'not found', 'La Dôle': 'not found', 'Adelboden': 'BE', 'Davos': 'GR', 'Lugano': 'TI', 'Zürich / Kloten': 'not found', 'St. Gallen': 'SG', 'Disentis / Sedrun': 'not found', 'Genève-Cointrin': 'not found', 'Visp': 'VS', 'Basel-Binningen': 'not found', 'Zermatt': 'VS', 'Chur': 'GR', 'Neuchâtel': 'NE', 'Interlaken': 'BE', 'Engelberg': 'OW', 'Luzern': 'LU', 'Säntis': 'not found', 'Pully': 'VD', 'Scuol': 'GR', 'Aigle': 'VD', 'Zürich / Fluntern': 'not found', 'Napf': 'not found', 'La Chaux-de-Fonds': 'NE', 'Sion': 'VS', 'Jungfraujoch Sphinx': 'not found', 'Aadorf / Tänikon': 'not found', 'Rünenberg': 'BL', 'Weissfluhjoch': 'not found', 'Samedan-Flugplatz': 'not found', 'Montana': 'not found', 'Glarus': 'GL', 'Buchs-Aarau': 'not found', 'Güttingen': 'TG', 'Vaduz': 'not found', 'Gütsch ob Andermatt': 'not found', 'San Bernardino': 'not found', 'Fahy': 'JU', 'Altdorf': 'not found', 'Stabio': 'TI', 'Locarno / Monti': 'not found', 'Cimetta': 'not found', 'Wynau': 'BE', 'Payerne': 'VD'

50

In [19]:
#Stations where the canton was found:

stations_with_canton_dict = {}  # Initialize an empty dictionary

for station, canton in station_canton_map.items():
    if canton != "not found":
        stations_with_canton_dict[station] = canton

print("\nStations with canton found (dictionary):")
print(stations_with_canton_dict)
len(stations_with_canton_dict)


Stations with canton found (dictionary):
{'Adelboden': 'BE', 'Davos': 'GR', 'Lugano': 'TI', 'St. Gallen': 'SG', 'Visp': 'VS', 'Zermatt': 'VS', 'Chur': 'GR', 'Neuchâtel': 'NE', 'Interlaken': 'BE', 'Engelberg': 'OW', 'Luzern': 'LU', 'Pully': 'VD', 'Scuol': 'GR', 'Aigle': 'VD', 'La Chaux-de-Fonds': 'NE', 'Sion': 'VS', 'Rünenberg': 'BL', 'Glarus': 'GL', 'Güttingen': 'TG', 'Fahy': 'JU', 'Stabio': 'TI', 'Wynau': 'BE', 'Payerne': 'VD', 'Schaffhausen': 'SH', 'Wädenswil': 'ZH'}


25

In [20]:
#Stations where the canton was not found:

stations_not_found = []  # Initialize an empty list

for station, canton in station_canton_map.items():  # Iterate through key-value pairs
  if canton == "not found":
    stations_not_found.append(station)

print(stations_not_found)  # Print the list of stations not found
len(stations_not_found)

['Pilatus', 'La Dôle', 'Zürich / Kloten', 'Disentis / Sedrun', 'Genève-Cointrin', 'Basel-Binningen', 'Säntis', 'Zürich / Fluntern', 'Napf', 'Jungfraujoch Sphinx', 'Aadorf / Tänikon', 'Weissfluhjoch', 'Samedan-Flugplatz', 'Montana', 'Buchs-Aarau', 'Vaduz', 'Gütsch ob Andermatt', 'San Bernardino', 'Altdorf', 'Locarno / Monti', 'Cimetta', 'Col du Gd-St-Bernard', 'Poschiavo / Robbia', 'Bern / Zollikofen', 'Le Moléson']


25

In [66]:
#Identify in which cantons stations have been found
station_cantons = []  # Initialize an empty list

for canton in station_canton_map.values():  # Iterate through the values (cantons)
  station_cantons.append(canton)  # Add each canton to the list

print(station_cantons)  # Print the list of cantons

['not found', 'not found', 'BE', 'GR', 'TI', 'not found', 'SG', 'not found', 'not found', 'VS', 'not found', 'VS', 'GR', 'NE', 'BE', 'OW', 'LU', 'not found', 'VD', 'GR', 'VD', 'not found', 'not found', 'NE', 'VS', 'not found', 'not found', 'BL', 'not found', 'not found', 'not found', 'GL', 'not found', 'TG', 'not found', 'not found', 'not found', 'JU', 'not found', 'TI', 'not found', 'not found', 'BE', 'VD', 'not found', 'not found', 'not found', 'SH', 'ZH', 'not found']


In [27]:
#Stations Missing or not found for following Cantons:

missing_cantons = []  # Initialize an empty list

for canton in cantons_list:
  if canton not in station_cantons:
    missing_cantons.append(canton)

print(missing_cantons)  # Print the list of missing cantons


['ZG', 'AI', 'SZ', 'AR', 'AG', 'BS', 'FR', 'GE', 'NW', 'UR', 'SO']


#Retrieval und Mapping der Stationskoordinaten auf Kantone


In [28]:
import osmium
import overpy
import json
import pandas as pd

# Load station coordinates from JSON file
with open('/content/drive/MyDrive/ie_scripting_datasets/station_coordinates.json', 'r') as f:
    station_coordinates = json.load(f)

# Create a list to store the data
station_data = []

# Function to get canton using Overpass API and OSM data
def get_canton(lat, lon):
    """Gets the canton name for a given latitude and longitude using Overpass API."""
    api = overpy.Overpass()
    query = f"""
    [out:json];
    is_in({lat},{lon});
    area._[admin_level=4][boundary=administrative];
    out center;
    """
    result = api.query(query)
    if result.areas:
        short_name = result.areas[0].tags.get("ISO3166-2", "N/A")  # Get canton code (ISO3166-2)
        full_name = result.areas[0].tags.get("name:de", "N/A")    # Get canton name in German
        return short_name, full_name  # Return both short name and full name
    else:
        return "N/A", "N/A"  # Return "N/A" if no canton is found

# Loop through each station and get its canton
for station_name, coordinates in station_coordinates.items():
    lat, lon = map(float, coordinates.split(','))  # Convert to float

    canton = get_canton(lat, lon)

    # Append data to the list
    station_data.append({
        "station_name": station_name,
        "coordinates": coordinates,
        "canton": canton
    })

    # Output station name, coordinates, and canton
    print(f'{station_name}, {coordinates}, {canton}')

# Create a Pandas DataFrame from the list
gemeinden = pd.DataFrame(station_data)

# Path to save the JSON file
file_path = '/content/drive/MyDrive/ie_scripting_datasets/station_cantons.json'

# Save the DataFrame to JSON
gemeinden.to_json(file_path, orient='records', force_ascii=False, indent=4)
print(f"Station data saved to {file_path}")

Pilatus, 46.9706835, 8.398695918732447, ('CH-NW', 'Nidwalden')
La Dôle, 46.4256492, 6.0995546, ('CH-VD', 'Waadt')
Adelboden, 46.4927213, 7.5587618, ('CH-BE', 'Bern')
Davos, 46.796198, 9.8236892, ('CH-GR', 'Graubünden')
Lugano, 46.0050102, 8.9520281, ('CH-TI', 'Tessin')
Zürich / Kloten, 47.443892950000006, 8.571472097092306, ('CH-ZH', 'Zürich')
St. Gallen, 47.425618, 9.3762397, ('CH-SG', 'St. Gallen')
Disentis / Sedrun, 46.703001900000004, 8.8186489317436, ('CH-GR', 'Graubünden')
Genève-Cointrin, 46.225539299999994, 6.100365627759709, ('CH-GE', 'Genf')
Visp, 46.2913113, 7.8811918, ('CH-VS', 'Wallis')
Basel-Binningen, 47.541255199999995, 7.582376595957297, ('CH-BL', 'Basel-Landschaft')
Zermatt, 46.0212076, 7.749254, ('CH-VS', 'Wallis')
Chur, 46.854747, 9.5264904, ('CH-GR', 'Graubünden')
Neuchâtel, 46.9895828, 6.9292641, ('CH-NE', 'Neuenburg')
Interlaken, 46.6855231, 7.8585139, ('CH-BE', 'Bern')
Engelberg, 46.8223497, 8.4043996, ('CH-OW', 'Obwalden')
Luzern, 47.0505452, 8.3054682, ('CH-LU

#Clean up von station_coordinates.json

In [29]:
# Load station coordinates from JSON file
with open('/content/drive/MyDrive/ie_scripting_datasets/station_cantons.json', 'r') as f:
    station_coordinates = json.load(f)

# Clean the canton values
for station_data in station_coordinates:
    canton_list = station_data['canton']
    if canton_list and canton_list[0].startswith('CH-'):
        canton_list[0] = canton_list[0][3:]  # Remove the "CH-" prefix from the short name

# Save the cleaned data to a new file
with open('/content/drive/MyDrive/ie_scripting_datasets/station_cantons_clean.json', 'w') as outfile:
    json.dump(station_coordinates, outfile, indent=4, ensure_ascii=False)

print("Cleaned station data saved to station_cantons_clean.json")

Cleaned station data saved to station_cantons_clean.json


#Anreicherung des Dataframe sunshine_data mit Koordinaten und Kantonskürzel der Stationsstandorte

In [31]:
print(sunshine_data)

               Station  Jan  Feb  Mär  Apr  Mai  Jun  Jul  Aug  Sep  Okt  Nov  \
0            Adelboden   75   97   84  112   99  116  185  213  106   81  NaN   
1                Aigle   73  102  111  125  118  150  244  256  129  112  NaN   
2              Altdorf   46   73   95  115  103  123  173  190  120   56  NaN   
3      Basel-Binningen   63   56  110  121  153  153  225  285  128   82  NaN   
4    Bern / Zollikofen   61   90  108  143  151  160  239  273  131   78  NaN   
..                 ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
695      Weissfluhjoch  156  162  227  239  233  124  125  251  212  226  205   
696              Wynau   54  104  180  272  286  175  233  255  195  116   35   
697            Zermatt  116  121  159  199  195  134  161  197  185  165  122   
698    Zürich / Kloten   39   92  183  273  288  171  218  265  185  112   53   
699  Zürich / Fluntern   53  104  178  280  280  158  193  259  204  126   79   

      Dez Total  Year  
0  

#Updated Dataframe für Sonnescheindaten inklusive Stationskoordinaten und Kantonskürzel gespeichert in einem .JSON

In [32]:
# Load station cantons data
with open('/content/drive/MyDrive/ie_scripting_datasets/station_cantons_clean.json', 'r', encoding='utf-8') as f:
    station_cantons = json.load(f)

# Create a dictionary to map station names to coordinates and cantons
station_data = {}
for station in station_cantons:
    station_data[station['station_name']] = {
        'coordinates': station['coordinates'],
        'canton_short_name': station['canton'][0],
        'canton_name': station['canton'][1]
    }

# Add coordinates and canton short names to sunshine data
sunshine_data_complete = sunshine_data.copy()

# Get coordinates and canton_short_name using the 'Station' column from the original DataFrame
coordinates = sunshine_data['Station'].map(lambda station: station_data.get(station, {}).get('coordinates'))
canton_short_name = sunshine_data['Station'].map(lambda station: station_data.get(station, {}).get('canton_short_name'))
canton_name = sunshine_data['Station'].map(lambda station: station_data.get(station, {}).get('canton_name'))

# Add the new columns to the DataFrame
sunshine_data_complete['coordinates'] = coordinates
sunshine_data_complete['canton_short_name'] = canton_short_name
sunshine_data_complete['canton_name'] = canton_name


# Save the complete data to a JSON file
# Explicitly specify the encoding as 'utf-8'
with open('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_complete_all_years.json', 'w', encoding='utf-8') as f:
    json.dump(sunshine_data_complete.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

In [33]:
print(sunshine_data_complete)


               Station  Jan  Feb  Mär  Apr  Mai  Jun  Jul  Aug  Sep  Okt  Nov  \
0            Adelboden   75   97   84  112   99  116  185  213  106   81  NaN   
1                Aigle   73  102  111  125  118  150  244  256  129  112  NaN   
2              Altdorf   46   73   95  115  103  123  173  190  120   56  NaN   
3      Basel-Binningen   63   56  110  121  153  153  225  285  128   82  NaN   
4    Bern / Zollikofen   61   90  108  143  151  160  239  273  131   78  NaN   
..                 ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
695      Weissfluhjoch  156  162  227  239  233  124  125  251  212  226  205   
696              Wynau   54  104  180  272  286  175  233  255  195  116   35   
697            Zermatt  116  121  159  199  195  134  161  197  185  165  122   
698    Zürich / Kloten   39   92  183  273  288  171  218  265  185  112   53   
699  Zürich / Fluntern   53  104  178  280  280  158  193  259  204  126   79   

      Dez Total  Year      

#Dataframe muss konsilidiert werden auf Jahr 2020 um Vergleichbarkeit mit Energiepreis und Mietpreis sicherzustellen

In [34]:
sunshine_data = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_complete_all_years.json')

sunshine_data_2020 = sunshine_data[sunshine_data['Year'] == 2020]
sunshine_data_2020.head()

#Show all Rows for visual check
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(sunshine_data_2020)



,Station,Jan,Feb,Mär,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez,Total,Year,coordinates,canton_short_name,canton_name
200,Adelboden,125,87,136,212,198,149,211,176,153,86,128,51.0,1710,2020,"46.4927213, 7.5587618",BE,Bern
201,Aigle,129,101,157,240,234,180,267,219,198,102,132,61.0,2020,2020,"46.3179006, 6.9688929",VD,Waadt
202,Altdorf,82,91,135,204,181,134,188,162,156,87,65,27.0,1511,2020,"46.8820732, 8.6413922",UR,Uri
203,Basel-Binningen,115,108,182,294,242,192,303,214,207,60,116,25.0,2057,2020,"47.541255199999995, 7.582376595957297",BL,Basel-Landschaft
204,Bern / Zollikofen,137,118,194,286,265,208,298,231,207,97,83,30.0,2155,2020,"46.9920954, 7.4565378",BE,Bern
205,Buchs-Aarau,66,88,172,288,239,193,286,207,187,72,56,11.0,1864,2020,"47.395399749999996, 8.071830123727523",AG,Aargau
206,La Chaux-de-Fonds,136,97,179,255,246,161,280,192,187,72,148,42.0,1993,2020,"47.1031321, 6.8325778",NE,Neuenburg
207,Chur,127,97,158,216,209,154,233,180,184,98,130,59.0,1844,2020,"46.854747, 9.5264904",GR,Graubünden
208,Cimetta,205,194,192,257,236,195,277,256,198,150,207,54.0,2419,2020,"46.2406177, 8.969494",TI,Tessin
209,Davos,136,105,170,231,190,124,211,175,185,115,160,65.0,1866,2020,"46.796198, 9.8236892",GR,Graubünden


#Speichern des konsolidierten Dataframe für 2020 als .JSON

In [35]:
# Define Sunshinedata DataFrame for year 2020
sunshine_data_2020_df = sunshine_data[sunshine_data['Year'] == 2020]

# Save to JSON file

with open('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(sunshine_data_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

#Ersetzen von "-" als NaN Values im Dataframe für Sunshine Data 2020



In [36]:
# Load data from JSON file, replacing hyphens with NaN after loading
sunshine_data_2020_df = pd.read_json(
    "/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_2020_df.json"
)

# Replace hyphens with NaN
sunshine_data_2020_df = sunshine_data_2020_df.replace('-', float('nan'))

#Show all Rows for visual check
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(sunshine_data_2020_df)

,Station,Jan,Feb,Mär,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez,Total,Year,coordinates,canton_short_name,canton_name
0,Adelboden,125,87,136,212,198,149,211,176,153,86,128,51.0,1710,2020,"46.4927213, 7.5587618",BE,Bern
1,Aigle,129,101,157,240,234,180,267,219,198,102,132,61.0,2020,2020,"46.3179006, 6.9688929",VD,Waadt
2,Altdorf,82,91,135,204,181,134,188,162,156,87,65,27.0,1511,2020,"46.8820732, 8.6413922",UR,Uri
3,Basel-Binningen,115,108,182,294,242,192,303,214,207,60,116,25.0,2057,2020,"47.541255199999995, 7.582376595957297",BL,Basel-Landschaft
4,Bern / Zollikofen,137,118,194,286,265,208,298,231,207,97,83,30.0,2155,2020,"46.9920954, 7.4565378",BE,Bern
5,Buchs-Aarau,66,88,172,288,239,193,286,207,187,72,56,11.0,1864,2020,"47.395399749999996, 8.071830123727523",AG,Aargau
6,La Chaux-de-Fonds,136,97,179,255,246,161,280,192,187,72,148,42.0,1993,2020,"47.1031321, 6.8325778",NE,Neuenburg
7,Chur,127,97,158,216,209,154,233,180,184,98,130,59.0,1844,2020,"46.854747, 9.5264904",GR,Graubünden
8,Cimetta,205,194,192,257,236,195,277,256,198,150,207,54.0,2419,2020,"46.2406177, 8.969494",TI,Tessin
9,Davos,136,105,170,231,190,124,211,175,185,115,160,65.0,1866,2020,"46.796198, 9.8236892",GR,Graubünden


#Speichern des Dataframe sunshine_data_2020_df mit NaN values für "-" Values

In [37]:
with open('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(sunshine_data_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

#Suchen der Fehlenden Kantone im Jahr 2020

In [38]:
#loop welcher eine liste min allen kantonsabkürzungen erstellt und als .json speichert


kantone = set()  # Initialize an empty set to store unique Kantone

with open('/content/drive/MyDrive/ie_scripting_datasets/gemeinden.json', 'r') as f:
    data = json.load(f)  # Load the JSON array

for item in data:  # Iterate through the objects in the array
    kantone.add(item['Kanton'])  # Extract 'Kanton' and add to set

kanton_list = list(kantone)  # Convert the set to a list

print(kanton_list)  # Print the list
print(len(kanton_list))

with open('/content/drive/MyDrive/ie_scripting_datasets/kanton_list.json', 'w', encoding='utf-8') as f:
  json.dump(kanton_list, f, ensure_ascii=False, indent=4)

['JU', 'ZG', 'AI', 'SZ', 'SH', 'AR', 'TG', 'GL', 'NE', 'AG', 'SG', 'LU', 'GR', 'VD', 'VS', 'BS', 'BE', 'BL', 'FR', 'GE', 'ZH', 'OW', 'NW', 'UR', 'SO', 'TI']
26


In [39]:
#Suchen der Fehlenden Kantone im Jahr 2020

# Load the kanton_list from JSON file
with open('/content/drive/MyDrive/ie_scripting_datasets/kanton_list.json', 'r') as f:
    kanton_list = json.load(f)

# Get unique values from the "canton_short_name" column
sunshine_cantons = set(sunshine_data_2020['canton_short_name'].unique())

# Find missing cantons
missing_cantons = [kanton for kanton in kanton_list if kanton not in sunshine_cantons]

# Print the missing cantons
print("Missing Cantons:", missing_cantons)

# Save the missing cantons to a JSON file
with open('/content/drive/MyDrive/ie_scripting_datasets/missing_cantons_2020.json', 'w', encoding='utf-8') as f:
    json.dump(missing_cantons, f)

Missing Cantons: ['ZG', 'AI', 'SZ', 'AR', 'BS', 'SO']


#Finden welche Kantone in der gesamten Wetterstationsliste nicht in vertreten sind

In [40]:

# Load the kanton_list from the JSON file
with open('/content/drive/MyDrive/ie_scripting_datasets/kanton_list.json', 'r') as f:
    kanton_list = json.load(f)

# Load station cantons from the JSON file
with open('/content/drive/MyDrive/ie_scripting_datasets/station_cantons_clean.json', 'r') as f:
    station_cantons_data = json.load(f)

# Extract the first element of the "canton" key for each station
station_cantons = set(item['canton'][0] for item in station_cantons_data)

# Find missing cantons
missing_cantons = [kanton for kanton in kanton_list if kanton not in station_cantons]

# Print the missing cantons
print("Missing Cantons:", missing_cantons)

Missing Cantons: ['ZG', 'AI', 'SZ', 'AR', 'BS', 'SO']


#Finden von Sonnenscheindaten für die fehlenden Kantone via Meteostat API
(https://dev.meteostat.net/python/monthly.html#api)

Das folgende skript wurde einfach mehrmals manuell mit den obigen Kantonsabkürzungen ausgeführt um einen überblick über die vorhandenen daten zu erhalten.

In [41]:
#Get weather stations in missing cantons
from meteostat import Stations

stations = Stations()
stations = stations.region('CH', 'BS')
stations_df = stations.fetch(2)#, sample=True)

print('Number of:', stations.count())
print("Station Data:",)

stations_df.head()

Number of: 1
Station Data:


,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end
id,,,,,,,,,,,,,,,
06601,Basel / Binningen,CH,BS,06601,<NA>,47.5411,7.5836,316.0,Europe/Zurich,NaT,NaT,1901-01-01,2024-11-30,1901-01-01,2022-01-01


In [42]:
from meteostat import Stations, Monthly

# Set time period
start = datetime(2020, 1, 1)
end = datetime(2020, 12, 31)

# Get Monthly data
sunshine_data_meteostat = Monthly('06601', start, end)
sunshine_data_meteostat_df = sunshine_data_meteostat.fetch()

# Show dataframe
sunshine_data_meteostat_df.head()

,tavg,tmin,tmax,prcp,wspd,pres,tsun
time,,,,,,,
2020-01-01,3.6,0.2,7.3,17.0,8.3,1027.3,6900.0
2020-02-01,7.6,3.2,12.8,70.0,13.3,1020.7,6480.0
2020-03-01,7.3,2.5,12.6,48.0,NaN,1018.1,10920.0
2020-04-01,13.5,6.1,20.6,17.0,5.7,1016.5,17640.0
2020-05-01,14.7,8.2,20.4,74.0,5.8,1019.2,14520.0


#**Feststellung**:
Für Kantone ['ZG','SO','SZ'] *können keine eindeutingen* Sonnenscheindaten gefunden werden. Anhand der meteostat API kann festgestellt werden, dass fehlende Daten zt. von kantonsnahen stationen übernommen werden. So zb bei ['AR', 'AI', 'BS'].

- AI --> "Säntis" --> SG
- AR --> "St. Gallen" --> SG
- BS --> "Basel / Binningen" --> BL

Definitiv ohne Werte sind:
- ZG
- SZ
- SO

Wir haben daher beschlossen Durchschnitte der angrenzenden Kantone zu verwenden um ebenfalls Daten für die Fehlenden Kantone zu erhalten.

#Hinzufügen der Durchschnittswerte für fehlende Sonnenscheindaten

In [43]:

# Load data from JSON file
sunshine_data_2020_df = pd.read_json(
    "/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_2020_df.json"
)

neighboring_cantons = {
    'ZG': ['LU', 'SZ', 'ZH'],
    'AR': ['AI', 'SG', 'GL'],
    'SO': ['BE', 'AG', 'BL'],
    'AI': ['AR', 'SG', 'GL'],
    'SZ': ['ZG', 'LU', 'UR'],
    'BS': ['BL'],
}

missing_cantons = ['ZG', 'AR', 'SO', 'AI', 'SZ', 'BS']
monthly_cols = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']

new_rows = []
for canton in missing_cantons:
    neighbors = neighboring_cantons[canton]
    neighbor_data = sunshine_data_2020_df[
        sunshine_data_2020_df['canton_short_name'].isin(neighbors)
    ]

    # Calculate average sunshine for each month and total
    avg_sunshine_monthly = neighbor_data[monthly_cols].mean()

    # Convert monthly averages to integers (0 decimal points)
    avg_sunshine_monthly = avg_sunshine_monthly.astype(int)

    avg_sunshine_total = avg_sunshine_monthly.sum()  # Calculate total from monthly averages

    new_row = {
        'Station': float('nan'),
        'coordinates': float('nan'),
        'canton_short_name': canton,
        'Year': 2020,
        'Total': avg_sunshine_total,
    }

    # Add monthly averages to the new row
    new_row.update(avg_sunshine_monthly.to_dict())

    new_rows.append(new_row)

# Initialize sunshine_data_2020_including_avg_df with sunshine_data_2020_df
sunshine_data_2020_including_avg_df = sunshine_data_2020_df.copy()

# Concatenate existing and new data
sunshine_data_2020_including_avg_df = pd.concat(
    [sunshine_data_2020_including_avg_df, pd.DataFrame(new_rows)],
    ignore_index=True,
)

#sunshine_data_2020_including_avg_df.head()

#Show all Rows for visual check
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(sunshine_data_2020_including_avg_df)

,Station,Jan,Feb,Mär,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez,Total,Year,coordinates,canton_short_name,canton_name
0,Adelboden,125,87,136,212,198,149.0,211,176,153,86,128,51.0,1710,2020,"46.4927213, 7.5587618",BE,Bern
1,Aigle,129,101,157,240,234,180.0,267,219,198,102,132,61.0,2020,2020,"46.3179006, 6.9688929",VD,Waadt
2,Altdorf,82,91,135,204,181,134.0,188,162,156,87,65,27.0,1511,2020,"46.8820732, 8.6413922",UR,Uri
3,Basel-Binningen,115,108,182,294,242,192.0,303,214,207,60,116,25.0,2057,2020,"47.541255199999995, 7.582376595957297",BL,Basel-Landschaft
4,Bern / Zollikofen,137,118,194,286,265,208.0,298,231,207,97,83,30.0,2155,2020,"46.9920954, 7.4565378",BE,Bern
5,Buchs-Aarau,66,88,172,288,239,193.0,286,207,187,72,56,11.0,1864,2020,"47.395399749999996, 8.071830123727523",AG,Aargau
6,La Chaux-de-Fonds,136,97,179,255,246,161.0,280,192,187,72,148,42.0,1993,2020,"47.1031321, 6.8325778",NE,Neuenburg
7,Chur,127,97,158,216,209,154.0,233,180,184,98,130,59.0,1844,2020,"46.854747, 9.5264904",GR,Graubünden
8,Cimetta,205,194,192,257,236,195.0,277,256,198,150,207,54.0,2419,2020,"46.2406177, 8.969494",TI,Tessin
9,Davos,136,105,170,231,190,124.0,211,175,185,115,160,65.0,1866,2020,"46.796198, 9.8236892",GR,Graubünden


#Speichern des neuen Dataframe sunshine_data_2020_df inklusive der fehlenden Kantone

In [44]:
sunshine_data_2020_df = sunshine_data_2020_including_avg_df

with open('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(sunshine_data_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

#Laden der Mietpreise as Dataframe und speichern als .json

In [45]:
filepath = '/content/drive/My Drive/ie_scripting_datasets/Mietpreise.xlsx'

# Create an empty dictionary to store the dataframes for each year
all_dfs = {}

# Read each sheet (year) into a separate dataframe and store it in the dictionary
xls = pd.ExcelFile(filepath)
for sheet_name in xls.sheet_names:
    all_dfs[sheet_name] = pd.read_excel(filepath, sheet_name=sheet_name, header=0,skiprows=1,
                                       names=['Canton', 'Total', '1-Zimmer', '2-Zimmer', '3-Zimmer',
                                              '4-Zimmer', '5-Zimmer', '6+-Zimmer'])
    all_dfs[sheet_name]['Year'] = sheet_name  # Add a 'Year' column

# Concatenate all dataframes into a single dataframe
mietpreise_allyears_df = pd.concat(all_dfs.values(), ignore_index=True)

# Display the dataframe
mietpreise_allyears_df.head()

with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_df.json', 'w', encoding='utf-8') as f:
  json.dump(mietpreise_allyears_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

# Display the dataframe
mietpreise_allyears_df.head()

,Canton,Total,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Year
0,Zürich,1597,989,1339,1538,1827,2262,2988,2020
1,Bern,1233,733,998,1193,1420,1722,2096,2020
2,Luzern,1364,745,1062,1325,1506,1826,2142,2020
3,Uri,1164,(641),(903),1077,1288,(14770),(1954),2020
4,Schwyz,1556,(782),1152,1483,1756,2113,(2517),2020


#Hinzufügen des "Canton_shortname" zu mietpreise_allyears_df.json wo möglich

In [46]:
from fuzzywuzzy import process
import pandas as pd
import json

def add_canton_shortname(mietpreise_df, sunshine_data_df):
    """
    Adds a 'Canton_shortname' column to mietpreise_df based on fuzzy matching
    with the 'canton_name' column in sunshine_data_df.

    Args:
        mietpreise_df: DataFrame containing rent data for all years.
        sunshine_data_df: DataFrame containing sunshine data for all years.

    Returns:
        A new DataFrame with an added 'Canton_shortname' column.
    """
    # Clean and standardize canton names
    def clean_name(name):
        if pd.isnull(name) or isinstance(name, int):  # Check if name is null or an integer:
            return None
        return name.replace('-', ' ').replace('/', ' ').strip().lower()

    mietpreise_df['Canton_cleaned'] = mietpreise_df['Canton'].apply(clean_name)
    sunshine_data_df['canton_name_cleaned'] = sunshine_data_df['canton_name'].apply(clean_name)

    # Create a mapping of canton_name_cleaned to canton_short_name
    canton_mapping = dict(zip(sunshine_data_df['canton_name_cleaned'], sunshine_data_df['canton_short_name']))

    # Unique canton names from sunshine_data_df for fuzzy matching
    sunshine_cantons = sunshine_data_df['canton_name_cleaned'].unique()

    # Function to match canton names
    def match_canton(canton_name):
        if pd.isnull(canton_name):
            return "NA"
        # Try direct mapping
        if canton_name in canton_mapping:
            return canton_mapping[canton_name]
        # Perform fuzzy matching if direct mapping fails
        match = process.extractOne(canton_name, sunshine_cantons, score_cutoff=85)
        if match:
            matched_name = match[0]
            return canton_mapping.get(matched_name, "NA")
        return "NA"

    # Apply the matching function
    mietpreise_df['Canton_shortname'] = mietpreise_df['Canton_cleaned'].apply(match_canton)

    # Drop temporary cleaned column
    mietpreise_df.drop(columns=['Canton_cleaned'], inplace=True)

    return mietpreise_df

# Load data
sunshine_data_path = '/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_complete_all_years.json'
mietpreise_data_path = '/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_df.json'

sunshine_data_df = pd.read_json(sunshine_data_path)
mietpreise_df = pd.read_json(mietpreise_data_path)

# Add Canton_shortname to mietpreise_df
mietpreise_allyears_canton_shortname_df = add_canton_shortname(mietpreise_df, sunshine_data_df)

# Save the updated DataFrame to a new JSON file with UTF-8 encoding
output_path = '/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_df.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(mietpreise_allyears_canton_shortname_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

print(f"Updated file saved as {output_path}")


Updated file saved as /content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_df.json


#Cleanup von mietpreise_allyears_canton_shortname_df.json

In [47]:
# Show all entries from mietpreise_allyears_canton_shortname_df.json that shows all entries where "Canton_shortname" is "N/A" or "NA"
# Load the JSON file into a DataFrame
mietpreise_allyears_canton_shortname_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_df.json')

# Filter the DataFrame
mietpreise_allyears_no_canton_shortname_df = mietpreise_allyears_canton_shortname_df[(mietpreise_allyears_canton_shortname_df['Canton_shortname'] == 'N/A') | (mietpreise_allyears_canton_shortname_df['Canton_shortname'] == 'NA')]

mietpreise_allyears_no_canton_shortname_df.head()

#Show all Rows for visual check
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(mietpreise_allyears_no_canton_shortname_df)


,Canton,Total,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Year,Canton_shortname
4,Schwyz,1556,(782),1152,1483,1756,2113,(2517),2020,NA
8,Zug,1824,890,1366,1686,2049,2598,3025,2020,NA
10,Solothurn,1196,693,944,1153,1373,1686,(2020),2020,NA
11,Basel-Stadt,1340,817,1099,1331,1717,2302,2816,2020,NA
14,Appenzell A.Rh.,1207,(554),(917),1119,1341,1498,(1963),2020,N/A


In [48]:
# Create a list from df mietpreise_allyears_no_canton_shortname_df with all entries from "Canton" where "Year" is not 2000
filtered_df = mietpreise_allyears_no_canton_shortname_df[mietpreise_allyears_no_canton_shortname_df['Year'] != 2000]

# Extract the "Canton" column and convert it to a list
mietpreise_no_canton_shortname_list = filtered_df['Canton'].tolist()
mietpreise_no_canton_shortname_list = set(mietpreise_no_canton_shortname_list)
mietpreise_no_canton_shortname_list = list(mietpreise_no_canton_shortname_list)
# Print the list (optional)
print(mietpreise_no_canton_shortname_list)

['Zug', 'Schwyz', 'Appenzell A.Rh.', 'Appenzell I.Rh.', 'Solothurn', 'Basel-Stadt']


In [49]:
#Mapping Dictionary for cantons without short name to use in correction loop
short_name_mapping_dict = {
    'Zug': 'ZG',
    'Basel-Stadt': 'BS',
    'Appenzell I.Rh.': 'AI',
    'Schwyz': 'SZ',
    'Solothurn': 'SO',
    'Appenzell A.Rh.': 'AR'
}

In [90]:
#loop to correct missing canton_short names to mietpreise_allyears_canton_shortname_df.json


# Assuming you loaded your data like this:
with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_df.json', 'r') as f:
  mietpreise_allyears_canton_shortname_clean_df = pd.read_json(f)

# Loop through each row to replace 'NA' or 'N/A' in 'Canton_shortname'
for index, row in mietpreise_allyears_canton_shortname_clean_df.iterrows():
    if row['Canton_shortname'] in ['NA', 'N/A']:
        canton_name = row['Canton']
        if canton_name in short_name_mapping_dict:
            mietpreise_allyears_canton_shortname_clean_df.at[index, 'Canton_shortname'] = short_name_mapping_dict[canton_name]
            print(f"Updated Canton_shortname for {canton_name} to {short_name_mapping_dict[canton_name]}")
        else:
            print(f"Warning: '{canton_name}' not found in mapping dictionary. Cannot update Canton_shortname.")


with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json', 'w', encoding='utf-8') as f:
  json.dump(mietpreise_allyears_canton_shortname_clean_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

#Optional Print:
print("Saved corrected version of mietpreise_allyears_canton_shortname_df as mietpreise_allyears_canton_shortname_clean_df.json")

Updated Canton_shortname for Schwyz to SZ
Updated Canton_shortname for Zug to ZG
Updated Canton_shortname for Solothurn to SO
Updated Canton_shortname for Basel-Stadt to BS
Updated Canton_shortname for Appenzell A.Rh. to AR
Updated Canton_shortname for Appenzell I.Rh. to AI
Updated Canton_shortname for Schwyz to SZ
Updated Canton_shortname for Zug to ZG
Updated Canton_shortname for Solothurn to SO
Updated Canton_shortname for Basel-Stadt to BS
Updated Canton_shortname for Appenzell A.Rh. to AR
Updated Canton_shortname for Appenzell I.Rh. to AI
Updated Canton_shortname for Schwyz to SZ
Updated Canton_shortname for Zug to ZG
Updated Canton_shortname for Solothurn to SO
Updated Canton_shortname for Basel-Stadt to BS
Updated Canton_shortname for Appenzell A.Rh. to AR
Updated Canton_shortname for Appenzell I.Rh. to AI
Updated Canton_shortname for Schwyz to SZ
Updated Canton_shortname for Zug to ZG
Updated Canton_shortname for Solothurn to SO
Updated Canton_shortname for Basel-Stadt to BS
Up

#Entfernen der () respektive Negative Mietpreise aus dem dataframe

In [115]:

def remove_brackets_from_dict(data):
    """
    Removes '(', ')', '*', and 'X' characters from string values
    in a nested dictionary or list of dictionaries.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, str):
                # Apply all replacements to the string
                cleaned_value = (value
                                 .replace('(', '')
                                 .replace(')', '')
                                 .replace('*', '')
                                 .replace('X', ''))
                data[key] = cleaned_value
            else:
                remove_brackets_from_dict(value)  # Recursively process nested values
    elif isinstance(data, list):
        for item in data:
            remove_brackets_from_dict(item)  # Recursively process list items

# Load the JSON data
with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Remove unwanted characters from values in the dictionary
remove_brackets_from_dict(data)

# Save the modified JSON data back to the file
with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# Reload the cleaned data into a pandas DataFrame if needed
mietpreise_allyears_canton_shortname_clean_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json')


In [116]:
# Save the modified JSON for mietpreise_allyears_canton_shortname_clean_df data back without brackets to the file

with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json', 'w', encoding='utf-8') as f:
  json.dump(mietpreise_allyears_canton_shortname_clean_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)
print("Saved corrected version of mietpreise_allyears_canton_shortname_df as mietpreise_allyears_canton_shortname_clean_df.json")


Saved corrected version of mietpreise_allyears_canton_shortname_df as mietpreise_allyears_canton_shortname_clean_df.json


#Anzeigen des Dataframe zur Datenkontrolle

In [117]:
mietpreise_allyears_canton_shortname_clean_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(mietpreise_allyears_canton_shortname_clean_df)

,Canton,Total,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Year,Canton_shortname
0,Zürich,1597,989,1339,1538,1827,2262,2988,2020,ZH
1,Bern,1233,733,998,1193,1420,1722,2096,2020,BE
2,Luzern,1364,745,1062,1325,1506,1826,2142,2020,LU
3,Uri,1164,641,903,1077,1288,14770,1954,2020,UR
4,Schwyz,1556,782,1152,1483,1756,2113,2517,2020,SZ
5,Obwalden,1356,756,996,1311,1509,1958,1963,2020,OW
6,Nidwalden,1494,705,1118,1426,1667,2032,2167,2020,NW
7,Glarus,1169,576,879,1136,1318,1602,1701,2020,GL
8,Zug,1824,890,1366,1686,2049,2598,3025,2020,ZG
9,Freiburg,1242,716,1000,1217,1444,1785,2041,2020,FR


#Eingränzung des Dataframe auf Jahr 2020

In [118]:

# Read the JSON file into a dataframe
mietpreise_allyears_canton_shortname_clean_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_allyears_canton_shortname_clean_df.json')

# Filter the dataframe for the year 2020
mietpreise_2020_df = mietpreise_allyears_canton_shortname_clean_df[mietpreise_allyears_canton_shortname_clean_df['Year'] == 2020]

# Display or use the filtered dataframe (df_2020) as needed

#mietpreise_2020_df.head()

#Display all rows for checking

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(mietpreise_2020_df)



,Canton,Total,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Year,Canton_shortname
0,Zürich,1597,989,1339,1538,1827,2262,2988,2020,ZH
1,Bern,1233,733,998,1193,1420,1722,2096,2020,BE
2,Luzern,1364,745,1062,1325,1506,1826,2142,2020,LU
3,Uri,1164,641,903,1077,1288,14770,1954,2020,UR
4,Schwyz,1556,782,1152,1483,1756,2113,2517,2020,SZ
5,Obwalden,1356,756,996,1311,1509,1958,1963,2020,OW
6,Nidwalden,1494,705,1118,1426,1667,2032,2167,2020,NW
7,Glarus,1169,576,879,1136,1318,1602,1701,2020,GL
8,Zug,1824,890,1366,1686,2049,2598,3025,2020,ZG
9,Freiburg,1242,716,1000,1217,1444,1785,2041,2020,FR


In [119]:
# Save the modified JSON for mietpreise_allyears_canton_shortname_clean_df data back without brackets to the file
with open('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(mietpreise_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)
print("Saved corrected version of mietpreise_allyears_canton_shortname_df as mietpreise_allyears_canton_shortname_clean_df.json")


Saved corrected version of mietpreise_allyears_canton_shortname_df as mietpreise_allyears_canton_shortname_clean_df.json


#Integrieren der Sonnenscheindauer mit Mietpreisen

In [120]:
#define the function to integrate the Dataframes

def integrate_dataframes(sunshine_data_2020_df, mietpreise_2020_df):
  """Integrates sunshine and rent dataframes based on canton name.

  Args:
    sunshine_data_2020_df: Pandas DataFrame containing sunshine data.
    mietpreise_2020_df: Pandas DataFrame containing rent data.

  Returns:
    A Pandas DataFrame with integrated data and desired columns.
  """

  # Read dataframes from JSON files
  sunshine_data_2020_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_2020_df.json')
  mietpreise_2020_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/mietpreise_2020_df.json')

  # Rename 'Canton_shortname' to 'canton_short_name' in mietpreise_2020_df for merging:
  mietpreise_2020_df = mietpreise_2020_df.rename(columns={"Canton_shortname": "canton_short_name"})  # Rename the 'Canton_shortname' column

  # Rename 'Total' columns in both dataframes before merging to avoid ambiguity:
  sunshine_data_2020_df = sunshine_data_2020_df.rename(columns={"Total": "Total_Sunhours"})
  mietpreise_2020_df = mietpreise_2020_df.rename(columns={"Total": "Total_Rent"})

  # Merge the dataframes:
  merged_df = pd.merge(sunshine_data_2020_df, mietpreise_2020_df, on='canton_short_name', how='inner')

  # Select desired columns:
  sunshine_data_and_mietpreise_2020_df = merged_df[['Station', 'Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez', 'Total_Sunhours', 'Year_x', 'coordinates', 'canton_short_name', 'canton_name', '1-Zimmer', '2-Zimmer', '3-Zimmer', '4-Zimmer', '5-Zimmer', '6+-Zimmer', 'Total_Rent']]

  return sunshine_data_and_mietpreise_2020_df

In [121]:
sunshine_data_2020_df.head()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(sunshine_data_2020_df)

,Station,Jan,Feb,Mär,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez,Total,Year,coordinates,canton_short_name,canton_name
0,Adelboden,125,87,136,212,198,149.0,211,176,153,86,128,51.0,1710,2020,"46.4927213, 7.5587618",BE,Bern
1,Aigle,129,101,157,240,234,180.0,267,219,198,102,132,61.0,2020,2020,"46.3179006, 6.9688929",VD,Waadt
2,Altdorf,82,91,135,204,181,134.0,188,162,156,87,65,27.0,1511,2020,"46.8820732, 8.6413922",UR,Uri
3,Basel-Binningen,115,108,182,294,242,192.0,303,214,207,60,116,25.0,2057,2020,"47.541255199999995, 7.582376595957297",BL,Basel-Landschaft
4,Bern / Zollikofen,137,118,194,286,265,208.0,298,231,207,97,83,30.0,2155,2020,"46.9920954, 7.4565378",BE,Bern
5,Buchs-Aarau,66,88,172,288,239,193.0,286,207,187,72,56,11.0,1864,2020,"47.395399749999996, 8.071830123727523",AG,Aargau
6,La Chaux-de-Fonds,136,97,179,255,246,161.0,280,192,187,72,148,42.0,1993,2020,"47.1031321, 6.8325778",NE,Neuenburg
7,Chur,127,97,158,216,209,154.0,233,180,184,98,130,59.0,1844,2020,"46.854747, 9.5264904",GR,Graubünden
8,Cimetta,205,194,192,257,236,195.0,277,256,198,150,207,54.0,2419,2020,"46.2406177, 8.969494",TI,Tessin
9,Davos,136,105,170,231,190,124.0,211,175,185,115,160,65.0,1866,2020,"46.796198, 9.8236892",GR,Graubünden


In [122]:
mietpreise_2020_df.head()

,Canton,Total,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Year,Canton_shortname
0,Zürich,1597,989,1339,1538,1827,2262,2988,2020,ZH
1,Bern,1233,733,998,1193,1420,1722,2096,2020,BE
2,Luzern,1364,745,1062,1325,1506,1826,2142,2020,LU
3,Uri,1164,641,903,1077,1288,14770,1954,2020,UR
4,Schwyz,1556,782,1152,1483,1756,2113,2517,2020,SZ


#Ausführen der Funktion und Speichern des Dataframe "sunshine_data_and_mietpreise_2020_df"

In [123]:
#run the function
sunshine_data_and_mietpreise_2020_df = integrate_dataframes(sunshine_data_2020_df, mietpreise_2020_df)

#save the file sunshine_data_and_mietpreise_2020_df.json
with open('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_and_mietpreise_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(sunshine_data_and_mietpreise_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

#Alle Zeilen des Dataframe "sunshine_data_and_mietpreise_2020_df" als Check

In [124]:


sunshine_data_and_mietpreise_2020_df.head()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sunshine_data_and_mietpreise_2020_df)

,Station,Jan,Feb,Mär,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez,Total_Sunhours,Year_x,coordinates,canton_short_name,canton_name,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Total_Rent
0,Adelboden,125,87,136,212,198,149.0,211,176,153,86,128,51.0,1710,2020,"46.4927213, 7.5587618",BE,Bern,733,998,1193,1420,1722,2096,1233
1,Aigle,129,101,157,240,234,180.0,267,219,198,102,132,61.0,2020,2020,"46.3179006, 6.9688929",VD,Waadt,817,1118,1377,1726,2079,2920,1384
2,Altdorf,82,91,135,204,181,134.0,188,162,156,87,65,27.0,1511,2020,"46.8820732, 8.6413922",UR,Uri,641,903,1077,1288,14770,1954,1164
3,Basel-Binningen,115,108,182,294,242,192.0,303,214,207,60,116,25.0,2057,2020,"47.541255199999995, 7.582376595957297",BL,Basel-Landschaft,739,1119,1350,1624,2059,2888,1418
4,Bern / Zollikofen,137,118,194,286,265,208.0,298,231,207,97,83,30.0,2155,2020,"46.9920954, 7.4565378",BE,Bern,733,998,1193,1420,1722,2096,1233
5,Buchs-Aarau,66,88,172,288,239,193.0,286,207,187,72,56,11.0,1864,2020,"47.395399749999996, 8.071830123727523",AG,Aargau,762,1111,1325,1555,1877,2149,1381
6,La Chaux-de-Fonds,136,97,179,255,246,161.0,280,192,187,72,148,42.0,1993,2020,"47.1031321, 6.8325778",NE,Neuenburg,606,780,938,1198,1509,2034,1002
7,Chur,127,97,158,216,209,154.0,233,180,184,98,130,59.0,1844,2020,"46.854747, 9.5264904",GR,Graubünden,731,1018,1254,1454,1624,1805,1242
8,Cimetta,205,194,192,257,236,195.0,277,256,198,150,207,54.0,2419,2020,"46.2406177, 8.969494",TI,Tessin,739,982,1199,1428,1780,2138,1192
9,Davos,136,105,170,231,190,124.0,211,175,185,115,160,65.0,1866,2020,"46.796198, 9.8236892",GR,Graubünden,731,1018,1254,1454,1624,1805,1242


#Hinzufügen des canton_short_name zu der Strompreis Liste für Mapping und speicher als .json

In [125]:

# Load the dataframe
strompreise_2020_df = pd.read_excel('/content/drive/MyDrive/ie_scripting_datasets/Median Strompreis per Kanton-full.xlsx', usecols=["Kanton", "Verbrauchsprofile typischer Haushalte", "Periode", "Produkt", "Total exkl. MWST (Rp./kWH)"])

# Canton mapping dictionary
canton_mapping = {
    "Zürich": "ZH",
    "Bern": "BE",
    "Luzern": "LU",
    "Uri": "UR",
    "Schwyz": "SZ",
    "Obwalden": "OW",
    "Nidwalden": "NW",
    "Glarus": "GL",
    "Zug": "ZG",
    "Freiburg": "FR",
    "Solothurn": "SO",
    "Basel-Stadt": "BS",
    "Basel-Landschaft": "BL",
    "Schaffhausen": "SH",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "St. Gallen": "SG",
    "Graubünden": "GR",
    "Aargau": "AG",
    "Thurgau": "TG",
    "Tessin": "TI",
    "Waadt": "VD",
    "Wallis": "VS",
    "Neuenburg": "NE",
    "Genf": "GE",
    "Jura": "JU",
}

# Create a new column 'canton_short_name'
strompreise_2020_df['canton_short_name'] = ''

# Iterate over the rows of the dataframe
for index, row in strompreise_2020_df.iterrows():
    canton_name = row['Kanton']
    if canton_name in canton_mapping:
        strompreise_2020_df.loc[index, 'canton_short_name'] = canton_mapping[canton_name]
    else:
        print(f"Warning: Canton name '{canton_name}' not found in mapping.")

# Specify the desired column order
column_order = ["Kanton", "canton_short_name", "Verbrauchsprofile typischer Haushalte", "Periode", "Produkt", "Total exkl. MWST (Rp./kWH)"]

# Reorder the columns in the dataframe
strompreise_2020_df = strompreise_2020_df[column_order]
strompreise_2020_df.head()

#Show all rows optionally
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(strompreise_2020_df)

#save the file strompreise_2020_df.json
with open('/content/drive/MyDrive/ie_scripting_datasets/strompreise_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(strompreise_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

strompreise_2020_df.head()

,Kanton,canton_short_name,Verbrauchsprofile typischer Haushalte,Periode,Produkt,Total exkl. MWST (Rp./kWH)
0,St. Gallen,SG,H4,2018,Günstigstes Produkt,18.400000
1,Wallis,VS,C3,2022,Günstigstes Produkt,17.489167
2,Tessin,TI,H5,2019,Günstigstes Produkt,17.292400
3,Schwyz,SZ,H4,2023,Günstigstes Produkt,24.661556
4,Zug,ZG,H4,2019,Günstigstes Produkt,18.203571


#Integration der Strompreise zum Dataframe sunshine_data_strom_mietpreise_2020_df mit Verbraucherprofil "H2" und "Standardprodukt" für Jahr 2020

In [126]:


# Read the Excel file into a pandas DataFrame
strompreise_2020_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/strompreise_2020_df.json')

# Read the JSON file into a pandas DataFrame
sunshine_data_and_mietpreise_2020_df = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_and_mietpreise_2020_df.json')

# Filter the strompreise_df DataFrame to only include the year 2020
strompreise_2020_df = strompreise_2020_df[(strompreise_2020_df['Periode'] == 2020) & (strompreise_2020_df['Verbrauchsprofile typischer Haushalte'] == 'H2') & (strompreise_2020_df['Produkt'] == 'Standardprodukt')]

# Merge the two DataFrames based on the canton name
sunshine_data_strom_mietpreise_2020_df = pd.merge(strompreise_2020_df, sunshine_data_and_mietpreise_2020_df, left_on='canton_short_name', right_on='canton_short_name', how='inner')

# Print the first 5 rows of the combined DataFrame to check the result (Optional)
#sunshine_data_strom_mietpreise_2020_df.head()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sunshine_data_strom_mietpreise_2020_df)

,Kanton,canton_short_name,Verbrauchsprofile typischer Haushalte,Periode,Produkt,Total exkl. MWST (Rp./kWH),Station,Jan,Feb,Mär,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez,Total_Sunhours,Year_x,coordinates,canton_name,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Total_Rent
0,Luzern,LU,H2,2020,Standardprodukt,13.596880,Luzern,99,115,161,273,229,178.0,253,223,178,80,76,32.0,1896,2020,"47.0505452, 8.3054682",Luzern,745,1062,1325,1506,1826,2142,1364
1,Appenzell Innerrhoden,AI,H2,2020,Standardprodukt,20.530857,None,116,83,148,240,200,145.0,218,185,164,84,124,59.0,1766,2020,None,None,,1033,1209,1598,1625,1666,1307
2,Basel-Stadt,BS,H2,2020,Standardprodukt,27.524674,None,116,103,175,293,242,188.0,295,213,200,65,118,26.0,2034,2020,None,None,817,1099,1331,1717,2302,2816,1340
3,St. Gallen,SG,H2,2020,Standardprodukt,21.898886,Säntis,162,87,172,242,178,120.0,187,178,177,102,197,89.0,1889,2020,"47.2493844, 9.3432366",St. Gallen,687,1002,1208,1375,1643,1952,1247
4,St. Gallen,SG,H2,2020,Standardprodukt,21.898886,St. Gallen,102,98,164,287,244,180.0,280,220,186,90,93,46.0,1990,2020,"47.425618, 9.3762397",St. Gallen,687,1002,1208,1375,1643,1952,1247
5,Graubünden,GR,H2,2020,Standardprodukt,22.769600,Chur,127,97,158,216,209,154.0,233,180,184,98,130,59.0,1844,2020,"46.854747, 9.5264904",Graubünden,731,1018,1254,1454,1624,1805,1242
6,Graubünden,GR,H2,2020,Standardprodukt,22.769600,Davos,136,105,170,231,190,124.0,211,175,185,115,160,65.0,1866,2020,"46.796198, 9.8236892",Graubünden,731,1018,1254,1454,1624,1805,1242
7,Graubünden,GR,H2,2020,Standardprodukt,22.769600,Disentis / Sedrun,128,97,154,244,204,122.0,232,193,189,87,128,40.0,1818,2020,"46.703001900000004, 8.8186489317436",Graubünden,731,1018,1254,1454,1624,1805,1242
8,Graubünden,GR,H2,2020,Standardprodukt,22.769600,Samedan-Flugplatz,155,125,164,219,195,147.0,220,177,162,124,176,68.0,1931,2020,"46.5299849, 9.8776444",Graubünden,731,1018,1254,1454,1624,1805,1242
9,Graubünden,GR,H2,2020,Standardprodukt,22.769600,San Bernardino,132,110,156,208,182,109.0,193,173,166,90,140,44.0,1701,2020,"46.523680799999994, 9.18635705",Graubünden,731,1018,1254,1454,1624,1805,1242


#Speichern des Dataframe als .json welcher nun Daten aus allen Datenquellen (Sonnenscheindauer / Mietpreise / Strom) enthält zur Visualisierung im Visualisierungsnotebook.

In [127]:
sunshine_data_strom_mietpreise_2020_df = sunshine_data_strom_mietpreise_2020_df

#save the file sunshine_data_and_mietpreise_2020_df.json
with open('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_strom_mietpreise_2020_df.json', 'w', encoding='utf-8') as f:
  json.dump(sunshine_data_strom_mietpreise_2020_df.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

In [128]:
cantons_short_names = pd.read_json('/content/drive/MyDrive/ie_scripting_datasets/sunshine_data_strom_mietpreise_2020_df.json')
cantons_short_names.head()


,Kanton,canton_short_name,Verbrauchsprofile typischer Haushalte,Periode,Produkt,Total exkl. MWST (Rp./kWH),Station,Jan,Feb,Mär,...,Year_x,coordinates,canton_name,1-Zimmer,2-Zimmer,3-Zimmer,4-Zimmer,5-Zimmer,6+-Zimmer,Total_Rent
0,Luzern,LU,H2,2020,Standardprodukt,13.596880,Luzern,99,115,161,...,2020,"47.0505452, 8.3054682",Luzern,745,1062,1325,1506,1826,2142,1364
1,Appenzell Innerrhoden,AI,H2,2020,Standardprodukt,20.530857,None,116,83,148,...,2020,None,None,,1033,1209,1598,1625,1666,1307
2,Basel-Stadt,BS,H2,2020,Standardprodukt,27.524674,None,116,103,175,...,2020,None,None,817,1099,1331,1717,2302,2816,1340
3,St. Gallen,SG,H2,2020,Standardprodukt,21.898886,Säntis,162,87,172,...,2020,"47.2493844, 9.3432366",St. Gallen,687,1002,1208,1375,1643,1952,1247
4,St. Gallen,SG,H2,2020,Standardprodukt,21.898886,St. Gallen,102,98,164,...,2020,"47.425618, 9.3762397",St. Gallen,687,1002,1208,1375,1643,1952,1247


#Final Check ob alle Kantone im dataframe vorhanden sind

In [129]:

canton_short_names = set(cantons_short_names['canton_short_name'])
num_unique_cantons = len(canton_short_names)

print(f"Number of unique cantons: {num_unique_cantons}")

Number of unique cantons: 26
